In [1]:
%%bash
pip install timm -q
pip install onnx -q
pip install onnxruntime -q

In [2]:
import os
import sys
import cv2
import timm
import onnx
import torch
import numpy as np
import onnxruntime as ort
import matplotlib.pyplot as plt

from torchvision import transforms

In [3]:
INPUT_PATH: str = "../input/fdsimages"
MODEL_PATH: str = "../input/fds-en4encoder-onnx"

REFERENCE_IMAGE_NAMES: list = [
    "Image_1_1.jpg",
    "Image_2_1.jpg",
    "Image_3_1.jpg",
    "Image_4_1.jpg",
]

In [4]:
def breaker() -> None:
    print("\n" + 50*"*" + "\n")


def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float: 
    return np.dot(a, b.reshape(-1, 1)) / (np.linalg.norm(a) * np.linalg.norm(b))


def get_image(path: str) -> np.ndarray: 
    return cv2.cvtColor(src=cv2.imread(path, cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB)


def get_reference_embeddings(models) -> list:
    reference_embeddings: list = []
    
    for i in range(len(models)):
        reference_image = get_image(os.path.join(INPUT_PATH, REFERENCE_IMAGE_NAMES[i]))
        reference_embeddings.append(models[i].infer(reference_image))
    
    return reference_embeddings

### **ONNX**

In [5]:
class Model(object):
    def __init__(self, model_type: int=1):
        self.ort_session = None
        self.size: int = 384

        if model_type == 1:
            self.path = os.path.join(MODEL_PATH, "onnx/I1T1-EN384AE.onnx")
            self.mean = [0.28106, 0.31696, 0.30282]
            self.std = [0.26783, 0.27980, 0.27595]
        
        elif model_type == 2:
            self.path = os.path.join(MODEL_PATH, "onnx/I2T1-EN384AE.onnx")
            self.mean = [0.33171, 0.40140, 0.42093]
            self.std = [0.23583, 0.24294, 0.24042]
        
        elif model_type == 3:
            self.path = os.path.join(MODEL_PATH, "onnx/I3T1-EN384AE.onnx")
            self.mean = [0.35717, 0.39790, 0.37881]
            self.std = [0.23648, 0.24145, 0.23653]
        
        elif model_type == 4:
            self.path = os.path.join(MODEL_PATH, "onnx/I4T1-EN384AE.onnx")
            self.mean = [0.41605, 0.47578, 0.45165]
            self.std = [0.20690, 0.20804, 0.19967]
        
        else:
            raise ValueError("Incorrect 'model_type'")

    def setup(self):
        model = onnx.load(self.path)
        onnx.checker.check_model(model)
        self.ort_session = ort.InferenceSession(self.path)

    def infer(self, image: np.ndarray) -> np.ndarray:
        image = image / 255
        image = cv2.resize(src=image, dsize=(self.size, self.size), interpolation=cv2.INTER_CUBIC).transpose(2, 0, 1)
        for i in range(image.shape[0]):
            image[i, :, :] = (image[i, :, :] - self.mean[i]) / self.std[i]
        image = np.expand_dims(image, axis=0)
        input = {self.ort_session.get_inputs()[0].name : image.astype("float32")}
        result = self.ort_session.run(None, input)
        return result[0].squeeze()

In [6]:
onnx_models: list = [
    Model(model_type=1), 
    Model(model_type=2), 
    Model(model_type=3), 
    Model(model_type=4)
]
for model in onnx_models: model.setup()
reference_embeddings = get_reference_embeddings(onnx_models)

filenames = sorted(os.listdir(INPUT_PATH))

breaker()
for filename in filenames:
    embeddings: list = []
    css: list = []
    for model in onnx_models:
        image = get_image(os.path.join(INPUT_PATH, filename))
        embeddings.append(model.infer(image))
    for i in range(len(onnx_models)): css.append(cosine_similarity(reference_embeddings[i], embeddings[i])[0])
    print(f"{filename} - CS1: {css[0]:.2f}, CS2: {css[1]:.2f}, CS3: {css[2]:.2f}, CS4: {css[3]:.2f}")
breaker()


**************************************************

Image_1_1.jpg - CS1: 1.00, CS2: 0.63, CS3: 0.49, CS4: 0.33
Image_1_2.jpg - CS1: 0.80, CS2: 0.69, CS3: 0.49, CS4: 0.50
Image_1_3.jpg - CS1: 0.89, CS2: 0.54, CS3: 0.42, CS4: 0.19
Image_1_4.jpg - CS1: 0.79, CS2: 0.62, CS3: 0.45, CS4: 0.40
Image_2_1.jpg - CS1: 0.66, CS2: 1.00, CS3: 0.45, CS4: 0.39
Image_2_2.jpg - CS1: 0.67, CS2: 0.85, CS3: 0.43, CS4: 0.42
Image_2_3.jpg - CS1: 0.58, CS2: 0.63, CS3: 0.37, CS4: 0.32
Image_2_4.jpg - CS1: 0.55, CS2: 0.76, CS3: 0.41, CS4: 0.35
Image_3_1.jpg - CS1: 0.33, CS2: 0.35, CS3: 1.00, CS4: 0.42
Image_3_2.jpg - CS1: 0.33, CS2: 0.26, CS3: 0.43, CS4: 0.11
Image_3_3.jpg - CS1: 0.30, CS2: 0.15, CS3: 0.53, CS4: 0.13
Image_3_4.jpg - CS1: 0.29, CS2: 0.20, CS3: 0.54, CS4: 0.05
Image_4_1.jpg - CS1: 0.36, CS2: 0.49, CS3: 0.20, CS4: 1.00
Image_4_2.jpg - CS1: 0.41, CS2: 0.51, CS3: 0.23, CS4: 0.75
Image_4_3.jpg - CS1: 0.35, CS2: 0.40, CS3: 0.23, CS4: 0.78
Image_4_4.jpg - CS1: 0.39, CS2: 0.45, CS3: 0.24, CS4: 0.79

**

### **PT**

In [7]:
class Model(torch.nn.Module):
    def __init__(self, model_type: int=1):
        super(Model, self).__init__()
        
        self.encoder = timm.create_model("efficientnet_b4", pretrained=False)
        self.encoder = torch.nn.Sequential(*[*self.encoder.children()][:-1])

    def forward(self, x):
        return self.encoder(x)

    
def get_features(model, path, transform, image):
    model.load_state_dict(torch.load(path, map_location=torch.device("cpu")))
    model.to(torch.device("cpu"))
    model.eval()
    
    image = cv2.resize(src=image, dsize=(384, 384), interpolation=cv2.INTER_AREA)
    with torch.no_grad(): embeddings = model(transform(image).to(torch.device("cpu")).unsqueeze(dim=0))
    return embeddings

In [8]:
reference_embeddings: list = []
    
model = Model().to(torch.device("cpu"))
paths = [
    "pt/I1T1-EN384AE.pt",
    "pt/I2T1-EN384AE.pt",
    "pt/I3T1-EN384AE.pt",
    "pt/I4T1-EN384AE.pt",
]
image_transforms = [
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.28106, 0.31696, 0.30282], [0.26783, 0.27980, 0.27595]),
    ]),
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.33171, 0.40140, 0.42093], [0.23583, 0.24294, 0.24042]),
    ]),
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.35717, 0.39790, 0.37881], [0.23648, 0.24145, 0.23653]),
    ]),
    transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.41605, 0.47578, 0.45165], [0.20690, 0.20804, 0.19967]),
    ]),
]

for i in range(len(paths)):
    image = get_image(os.path.join(INPUT_PATH, REFERENCE_IMAGE_NAMES[i]))
    reference_embeddings.append(get_features(model, os.path.join(MODEL_PATH, paths[i]), image_transforms[i], image))


filenames = sorted(os.listdir(INPUT_PATH))

breaker()
for filename in filenames:
    embeddings: list = []
    css: list = []
    for i in range(len(paths)):
        image = get_image(os.path.join(INPUT_PATH, filename))
        embeddings.append(get_features(model, os.path.join(MODEL_PATH, paths[i]), image_transforms[i], image))
    for i in range(len(paths)): css.append(torch.nn.CosineSimilarity()(reference_embeddings[i], embeddings[i]).item())
    print(f"{filename} - CS1: {css[0]:.2f}, CS2: {css[1]:.2f}, CS3: {css[2]:.2f}, CS4: {css[3]:.2f}")
breaker()


**************************************************

Image_1_1.jpg - CS1: 1.00, CS2: 0.63, CS3: 0.48, CS4: 0.33
Image_1_2.jpg - CS1: 0.80, CS2: 0.69, CS3: 0.49, CS4: 0.51
Image_1_3.jpg - CS1: 0.89, CS2: 0.54, CS3: 0.42, CS4: 0.19
Image_1_4.jpg - CS1: 0.79, CS2: 0.62, CS3: 0.46, CS4: 0.40
Image_2_1.jpg - CS1: 0.66, CS2: 1.00, CS3: 0.45, CS4: 0.38
Image_2_2.jpg - CS1: 0.67, CS2: 0.85, CS3: 0.43, CS4: 0.42
Image_2_3.jpg - CS1: 0.58, CS2: 0.63, CS3: 0.37, CS4: 0.32
Image_2_4.jpg - CS1: 0.55, CS2: 0.76, CS3: 0.41, CS4: 0.35
Image_3_1.jpg - CS1: 0.34, CS2: 0.35, CS3: 1.00, CS4: 0.42
Image_3_2.jpg - CS1: 0.33, CS2: 0.26, CS3: 0.43, CS4: 0.11
Image_3_3.jpg - CS1: 0.30, CS2: 0.15, CS3: 0.53, CS4: 0.13
Image_3_4.jpg - CS1: 0.29, CS2: 0.19, CS3: 0.54, CS4: 0.05
Image_4_1.jpg - CS1: 0.37, CS2: 0.50, CS3: 0.20, CS4: 1.00
Image_4_2.jpg - CS1: 0.41, CS2: 0.51, CS3: 0.22, CS4: 0.75
Image_4_3.jpg - CS1: 0.35, CS2: 0.40, CS3: 0.23, CS4: 0.78
Image_4_4.jpg - CS1: 0.39, CS2: 0.45, CS3: 0.24, CS4: 0.78

**